In [30]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/movies_metadata.csv')

In [31]:
# Print all the features (or columns) of the DataFrame
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [32]:
# Only keep those features that we require
df = df[['title', 'genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]
df.head()

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [33]:
# Convert release_date into pandas datetime format
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_date'].head()

0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10
Name: release_date, dtype: datetime64[ns]

In [34]:
# Extract year from datetime
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [35]:
# Helper function to convert Nat to 0 and all other years to integers
def convert_int(x):
    try:
        return int(x)
    except:
        return 0


In [36]:
# Apply convert_int to the year feature
df['year'] = df['year'].apply(convert_int)

In [37]:
# Drop the release_date column 
df = df.drop('release_date', axis=1)



In [38]:
# Display the dateframe
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995


In [39]:
# Print genre of the first movie
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [40]:
# Import the literal_eval function from ast
from ast import literal_eval

In [41]:
# Define a strinfield list and print its type
a = "[1, 2, 3]"
type(a)

str

In [42]:
# Apply literal_eval and output type
b = literal_eval(a)
type(b)

list

In [43]:
# Convert all NaN into stringfield empty lists
df['genres'] = df['genres'].fillna('[]')

In [44]:
# Apply literal_eval to convert to the list object
df['genres'] = df['genres'].apply(literal_eval)

In [45]:
# Convert list of dictionaries to a list of string
df['genres'] = df['genres'].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])

In [46]:
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995


In [47]:
# Create a new feature by exploding genres
s = df.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)

In [48]:
s.head()

0    animation
0       comedy
0       family
1    adventure
1      fantasy
dtype: object

In [49]:
# Name the new feature as "genre"
s.name = 'genre'

In [50]:
s.head()

0    animation
0       comedy
0       family
1    adventure
1      fantasy
Name: genre, dtype: object

In [51]:
# Create a new dataframe gen_df which by dropping the old 'genres' feature and adding the new 'genre'
gen_df = df.drop('genres', axis=1).join(s)

In [52]:
# Print the head of the gen_df
gen_df.head()

,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,animation
0,Toy Story,81.0,7.7,5415.0,1995,comedy
0,Toy Story,81.0,7.7,5415.0,1995,family
1,Jumanji,104.0,6.9,2413.0,1995,adventure
1,Jumanji,104.0,6.9,2413.0,1995,fantasy


In [53]:
def build_chart(gen_df, percentile=0.8):
    #Ask for preferred genres
    print("Input preferred genre")
    genre = input()
    
    #Ask for lower limit of duration
    print("Input shortest duration")
    low_time = int(input())
    
    #Ask for upper limit of duration
    print("Input longest duration")
    high_time = int(input())
    
    #Ask for lower limit of timeline
    print("Input earliest year")
    low_year = int(input())
    
    #Ask for upper limit of timeline
    print("Input latest year")
    high_year = int(input())
    
    #Define a new movies variable to store the preferred movies. Copy the contents of gen_df to movies
    movies = gen_df.copy()
    
    #Filter based on the condition
    movies = movies[(movies['genre'] == genre) & 
                    (movies['runtime'] >= low_time) & 
                    (movies['runtime'] <= high_time) & 
                    (movies['year'] >= low_year) & 
                    (movies['year'] <= high_year)]
    
    #Compute the values of C and m for the filtered movies
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    
    #Only consider movies that have higher than m votes. Save this in a new dataframe q_movies
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    
    #Calculate score using the IMDB formula
    q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) 
                                       + (m/(m+x['vote_count']) * C)
                                       ,axis=1)

    #Sort movies in descending order of their scores
    q_movies = q_movies.sort_values('score', ascending=False)
    
    return q_movies
    

In [54]:
# Generate chart for top animated movies and display top 5
build_chart(gen_df).head()

Input preferred genre
animation
Input shortest duration
30
Input longest duration
120
Input earliest year
1990
Input latest year
2005


,title,runtime,vote_average,vote_count,year,genre,score
9698,Howl's Moving Castle,119.0,8.2,2049.0,2004,animation,7.994823
359,The Lion King,89.0,8.0,5520.0,1994,animation,7.926672
0,Toy Story,81.0,7.7,5415.0,1995,animation,7.637500
6232,Finding Nemo,100.0,7.6,6292.0,2003,animation,7.549423
546,The Nightmare Before Christmas,76.0,7.6,2135.0,1993,animation,7.460500


In [55]:
# Convert the clean (non-exploded) dataframe df into a CSV file and save it in the data folder 
# Set parameter index to False, as the index of the DataFrame has no inherent meaning. 
df.to_csv('../data/metadata_clean1.csv', index=False)